In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0
import wandb
from tensorflow.keras import mixed_precision

# Enable mixed precision training
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [4]:
import os

In [5]:
tf.data.experimental.enable_debug_mode = False

In [7]:
# Define directories
train_dir = 'D:/food_not_food/data/train'
test_dir = 'D:/food_not_food/data/test'

In [8]:
# Get the number of classes from the training directory
num_classes = len(os.listdir(train_dir))

In [9]:
# Data preparation
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    seed=42,
    image_size=(224, 224),
    batch_size=64,
    label_mode='categorical'  # Specify categorical labels
)

Found 11802 files belonging to 2 classes.


In [10]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=42,
    image_size=(224, 224),
    batch_size=64,
    label_mode='categorical'  # Specify categorical labels
)

Found 2248 files belonging to 2 classes.


In [11]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

In [12]:
# Apply data augmentation to the training dataset
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))

In [13]:
# Prefetch data for better performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [14]:
# Define the model
base_model = EfficientNetV2B0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

24274472/24274472 [==============================] - 6s 0us/step


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
!wandb login 6831aadea925e84318b1e9b4086360f219939fd5

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Avnish\.netrc


In [17]:
# Initialize WandB
wandb.init(project="food_not_food", 
           config={"learning_rate": 0.001, 
                   "architecture": "EfficientNetV2B2",
                   "dataset": "Food-101 and filtered ImageNet-1k",
                   "epochs": 10})
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: avnishs17 (sukuna). Use `wandb login --relogin` to force relogin


In [18]:
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10


In [19]:
# Log metrics to WandB after each epoch
for epoch in range(10):
    wandb.log({
        "epoch": epoch,
        "loss": history.history["loss"][epoch],
        "accuracy": history.history["accuracy"][epoch],
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_accuracy"][epoch]
    })

In [20]:
model.save('/kaggle/working/', save_format='tf')


ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=tf

In [ ]:
loaded_model = tf.keras.models.load_model('/kaggle/working/')


In [1]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

NameError: name 'tf' is not defined

In [ ]:
# Save the model
with open('foo_nt_food.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.preprocessing import image
# import numpy as np

# # Path to the saved model
# model_path = 'path_to_my_model'

# # Load the saved model
# model = tf.keras.models.load_model(model_path)

# # Image path
# image_path = '/kaggle/input/mini-dataset-food-101-and-imagnet/data/test/food/acorn squash/54.jpg'

# # Load and preprocess the image
# img = image.load_img(image_path, target_size=(224, 224))  # Adjust the size to match your model's expected input
# img_array = image.img_to_array(img)
# img_array = np.expand_dims(img_array, axis=0)  # Create a batch
# img_array /= 255.0  # Normalize to [0,1] (if that's what your model expects)

# # Make a prediction
# predictions = model.predict(img_array)
# predicted_class = np.argmax(predictions, axis=1)

# # Output the result
# print("Predicted class:", predicted_class)


In [ ]:
# tensorflowjs_converter --input_format=keras path_to_my_model/ path_to_tfjs_model/
